A demonstration of converting a [Python vtreat](https://github.com/WinVector/vtreat) transformation into a [data algebra](https://github.com/WinVector/data_algebra) pipeline, which can then in turn be converted to SQL queries.
[R vtreat](https://winvector.github.io/vtreat/) already has similar functionality with [as_rquery_plan()](https://winvector.github.io/vtreat/reference/as_rquery_plan.html).

Let's demonstrate this with a simple problem.

First we import our modules.

In [13]:

import pandas as pd

from data_algebra.data_ops import *
import data_algebra.SQLite
import data_algebra.test_util
import vtreat
from vtreat.vtreat_db_adapter import as_data_algebra_pipeline

In [14]:
# Data from:
# https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008

data_all = pd.read_csv("diabetes_head.csv")
n = data_all.shape[0]
data_all['orig_index'] = range(n)
d_train = data_all.loc[range(n-5), :].reset_index(inplace=False, drop=True)
d_app = data_all.loc[range(n-5, n)].reset_index(inplace=False, drop=True)

In [15]:
outcome_name = "readmitted"
cols_to_copy = ["orig_index", "encounter_id", "patient_nbr"] + [outcome_name]
vars = ['time_in_hospital', 'weight']
columns = vars + cols_to_copy

d_train.loc[:, columns]


,time_in_hospital,weight,orig_index,encounter_id,patient_nbr,readmitted
0,1,NaN,0,2278392,8222157,False
1,2,NaN,1,64410,86047875,False
2,2,NaN,2,500364,82442376,False
3,3,NaN,3,35754,82637451,False
4,4,NaN,4,55842,84259809,False
...,...,...,...,...,...,...
990,7,NaN,990,8834994,66138012,False
991,6,NaN,991,8851308,71896626,False
992,11,NaN,992,8856594,79469091,False
993,12,NaN,993,8857092,92251359,True


,time_in_hospital,weight,orig_index,encounter_id,patient_nbr,readmitted
0,1,NaN,0,2278392,8222157,False
1,2,NaN,1,64410,86047875,False
2,2,NaN,2,500364,82442376,False
3,3,NaN,3,35754,82637451,False
4,4,NaN,4,55842,84259809,False
...,...,...,...,...,...,...
990,7,NaN,990,8834994,66138012,False
991,6,NaN,991,8851308,71896626,False
992,11,NaN,992,8856594,79469091,False
993,12,NaN,993,8857092,92251359,True


In [16]:
treatment = vtreat.BinomialOutcomeTreatment(
    cols_to_copy=cols_to_copy,
    outcome_name=outcome_name,
    outcome_target=True,
    params=vtreat.vtreat_parameters(
        {"sparse_indicators": False, "filter_to_recommended": False,}
    ),
)
d_train_treated = treatment.fit_transform(d_train.loc[:, columns])

d_app_treated = treatment.transform(d_app.loc[:, columns])

d_app_treated

,orig_index,encounter_id,patient_nbr,readmitted,weight_is_bad,time_in_hospital,weight_logit_code,weight_prevalence_code,weight_lev__NA_
0,995,8860284,94419315,False,1.0,3.0,0.005825,0.99397,1.0
1,996,8860944,338247,False,1.0,4.0,0.005825,0.99397,1.0
2,997,8864718,695439,False,1.0,1.0,0.005825,0.99397,1.0
3,998,8866632,103586670,False,0.0,6.0,0.000000,0.00000,0.0
4,999,8867106,4988970,False,1.0,9.0,0.005825,0.99397,1.0


,orig_index,encounter_id,patient_nbr,readmitted,weight_is_bad,time_in_hospital,weight_logit_code,weight_prevalence_code,weight_lev__NA_
0,995,8860284,94419315,False,1.0,3.0,0.005825,0.99397,1.0
1,996,8860944,338247,False,1.0,4.0,0.005825,0.99397,1.0
2,997,8864718,695439,False,1.0,1.0,0.005825,0.99397,1.0
3,998,8866632,103586670,False,0.0,6.0,0.000000,0.00000,0.0
4,999,8867106,4988970,False,1.0,9.0,0.005825,0.99397,1.0


In [17]:
transform_as_data = treatment.description_matrix()

transform_as_data

,treatment_class,treatment,orig_var,variable,value,replacement
0,IndicateMissingTransform,missing_indicator,weight,weight_is_bad,_NA_,1.000000e+00
1,CleanNumericTransform,clean_copy,time_in_hospital,time_in_hospital,_NA_,4.803015e+00
2,MappedCodeTransform,logit_code,weight,weight_logit_code,[0-25),-1.110223e-16
3,MappedCodeTransform,logit_code,weight,weight_logit_code,[50-75),-1.110223e-16
4,MappedCodeTransform,logit_code,weight,weight_logit_code,[75-100),-2.253714e+00
5,MappedCodeTransform,logit_code,weight,weight_logit_code,_NA_,5.825087e-03
6,MappedCodeTransform,prevalence_code,weight,weight_prevalence_code,[0-25),1.005025e-03
7,MappedCodeTransform,prevalence_code,weight,weight_prevalence_code,[50-75),1.005025e-03
8,MappedCodeTransform,prevalence_code,weight,weight_prevalence_code,[75-100),4.020101e-03
9,MappedCodeTransform,prevalence_code,weight,weight_prevalence_code,_NA_,9.939698e-01


,treatment_class,treatment,orig_var,variable,value,replacement
0,IndicateMissingTransform,missing_indicator,weight,weight_is_bad,_NA_,1.000000e+00
1,CleanNumericTransform,clean_copy,time_in_hospital,time_in_hospital,_NA_,4.803015e+00
2,MappedCodeTransform,logit_code,weight,weight_logit_code,[0-25),-1.110223e-16
3,MappedCodeTransform,logit_code,weight,weight_logit_code,[50-75),-1.110223e-16
4,MappedCodeTransform,logit_code,weight,weight_logit_code,[75-100),-2.253714e+00
5,MappedCodeTransform,logit_code,weight,weight_logit_code,_NA_,5.825087e-03
6,MappedCodeTransform,prevalence_code,weight,weight_prevalence_code,[0-25),1.005025e-03
7,MappedCodeTransform,prevalence_code,weight,weight_prevalence_code,[50-75),1.005025e-03
8,MappedCodeTransform,prevalence_code,weight,weight_prevalence_code,[75-100),4.020101e-03
9,MappedCodeTransform,prevalence_code,weight,weight_prevalence_code,_NA_,9.939698e-01


In [18]:
ops = as_data_algebra_pipeline(
    source=descr(d_app=d_app.loc[:, columns]),
    vtreat_descr=transform_as_data,
    treatment_table_name='transform_as_data',
)

print(ops)

(
    TableDescription(
        table_name="d_app",
        column_names=[
            "time_in_hospital",
            "weight",
            "orig_index",
            "encounter_id",
            "patient_nbr",
            "readmitted",
        ],
    )
    .extend(
        {
            "weight_is_bad": "(weight.is_bad()).if_else(1.0, 0.0)",
            "weight_lev__NA_": "(weight.coalesce('_NA_') == '_NA_').if_else(1.0, 0.0)",
        }
    )
    .extend({"weight": "weight.coalesce('_NA_')"})
    .natural_join(
        b=TableDescription(
            table_name="transform_as_data",
            column_names=[
                "treatment_class",
                "treatment",
                "orig_var",
                "variable",
                "value",
                "replacement",
            ],
        )
        .select_rows(
            "(treatment_class == 'MappedCodeTransform') and (orig_var == 'weight') and (variable == 'weight_logit_code')"
        )
        .extend({"weight": "

In [19]:
transformed = ops.eval({
    'd_app': d_app.loc[:, columns],
    'transform_as_data': transform_as_data})

transformed

,time_in_hospital,orig_index,encounter_id,patient_nbr,readmitted,weight_is_bad,weight_lev__NA_,weight_logit_code,weight_prevalence_code
0,3,995,8860284,94419315,False,1.0,1.0,0.005825,0.99397
1,4,996,8860944,338247,False,1.0,1.0,0.005825,0.99397
2,1,997,8864718,695439,False,1.0,1.0,0.005825,0.99397
3,6,998,8866632,103586670,False,0.0,0.0,0.000000,0.00000
4,9,999,8867106,4988970,False,1.0,1.0,0.005825,0.99397


,time_in_hospital,orig_index,encounter_id,patient_nbr,readmitted,weight_is_bad,weight_lev__NA_,weight_logit_code,weight_prevalence_code
0,3,995,8860284,94419315,False,1.0,1.0,0.005825,0.99397
1,4,996,8860944,338247,False,1.0,1.0,0.005825,0.99397
2,1,997,8864718,695439,False,1.0,1.0,0.005825,0.99397
3,6,998,8866632,103586670,False,0.0,0.0,0.000000,0.00000
4,9,999,8867106,4988970,False,1.0,1.0,0.005825,0.99397


In [20]:

assert data_algebra.test_util.equivalent_frames(transformed, d_app_treated)

In [21]:
db_handle = data_algebra.SQLite.example_handle()

sql = db_handle.to_sql(ops)
print(sql)

-- data_algebra SQL https://github.com/WinVector/data_algebra
--  dialect: SQLiteModel
--       string quote: '
--   identifier quote: "
WITH
 "extend_0" AS (
  SELECT  -- .extend({ 'weight_is_bad': '(weight.is_bad()).if_else(1.0, 0.0)', 'weight_lev__NA_': "(weight.coalesce('_NA_') == '_NA_').if_else(1.0, 0.0)"})
   "orig_index" ,
   "readmitted" ,
   "time_in_hospital" ,
   "patient_nbr" ,
   "encounter_id" ,
   "weight" ,
   CASE WHEN is_bad("weight") THEN 1.0 WHEN NOT is_bad("weight") THEN 0.0 ELSE NULL END AS "weight_is_bad" ,
   CASE WHEN (COALESCE("weight", '_NA_') = '_NA_') THEN 1.0 WHEN NOT (COALESCE("weight", '_NA_') = '_NA_') THEN 0.0 ELSE NULL END AS "weight_lev__NA_"
  FROM
   "d_app"
 ) ,
 "extend_1" AS (
  SELECT  -- .extend({ 'weight': "weight.coalesce('_NA_')"})
   "orig_index" ,
   "readmitted" ,
   "time_in_hospital" ,
   "weight_lev__NA_" ,
   "patient_nbr" ,
   "weight_is_bad" ,
   "encounter_id" ,
   COALESCE("weight", '_NA_') AS "weight"
  FROM
   "extend_0"
 ) ,


In [22]:
db_handle.insert_table(d_app.loc[:, columns], table_name='d_app')
db_handle.insert_table(transform_as_data, table_name='transform_as_data')

db_handle.execute('CREATE TABLE res AS ' + sql)

res_db = db_handle.read_query('SELECT * FROM res ORDER BY orig_index LIMIT 10')

res_db

,orig_index,readmitted,weight_lev__NA_,patient_nbr,weight_is_bad,encounter_id,weight_logit_code,weight_prevalence_code,time_in_hospital
0,995,0,1.0,94419315,1.0,8860284,0.005825,0.99397,3
1,996,0,1.0,338247,1.0,8860944,0.005825,0.99397,4
2,997,0,1.0,695439,1.0,8864718,0.005825,0.99397,1
3,998,0,0.0,103586670,0.0,8866632,0.000000,0.00000,6
4,999,0,1.0,4988970,1.0,8867106,0.005825,0.99397,9


,orig_index,readmitted,weight_lev__NA_,patient_nbr,weight_is_bad,encounter_id,weight_logit_code,weight_prevalence_code,time_in_hospital
0,995,0,1.0,94419315,1.0,8860284,0.005825,0.99397,3
1,996,0,1.0,338247,1.0,8860944,0.005825,0.99397,4
2,997,0,1.0,695439,1.0,8864718,0.005825,0.99397,1
3,998,0,0.0,103586670,0.0,8866632,0.000000,0.00000,6
4,999,0,1.0,4988970,1.0,8867106,0.005825,0.99397,9


In [23]:
assert data_algebra.test_util.equivalent_frames(res_db, d_app_treated)

In [24]:
db_handle.close()
